In [ ]:
import pandas as pd
from transformers import RobertaForSequenceClassification, RobertaTokenizer
import torch

In [ ]:
# Loading pre-trained tokenizer and model for RoBERTa
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Switching environment to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Importing the dataset
url = 'https://raw.githubusercontent.com/yamanalab/ec-darkpattern/master/dataset/dataset.tsv'
df = pd.read_csv(url, sep="\t")

In [ ]:
# Removing irrelevant columns
df = df[['text', 'label']]

In [ ]:
# Encoding the dataset
encoded = tokenizer(df.text.tolist(), truncation=True, padding=True)

In [ ]:
# Creating torch dataset
input_ids = torch.tensor(encoded['input_ids'])
attention_masks = torch.tensor(encoded['attention_mask'])
labels = torch.tensor(df.label.tolist())
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)


In [ ]:
# Training the RoBERTa model
train_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, pin_memory=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)

In [ ]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        batch = [t.to(device) for t in batch]
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
# Save the trained RoBERTa model
torch.save(model.state_dict(), "roberta_model.pt")

In [ ]:
# Load the trained RoBERTa model
loaded_model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
loaded_model.load_state_dict(torch.load("roberta_model.pt"))
loaded_model.eval()